In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
news_url = "https://mars.nasa.gov/news/"
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
weather_url = "https://twitter.com/marswxreport?lang=en"
facts_url = "http://space-facts.com/mars/)"
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

#### Scrape NASA website using requests to collect latest News title and paragraph text 

In [6]:
news_response = requests.get(news_url)

In [7]:
soup = bs(news_response.text, 'html.parser')
#print(soup.prettify())

In [8]:
news_title = soup.find('div', class_="content_title").text.strip()
print(news_title)

Opportunity Hunkers Down During Dust Storm


In [9]:
news_para = soup.find('div', class_='rollover_description_inner').text.strip()
print(news_para)

It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home.


#### Navigate the JPL site using Splinter to find the current Featured Image  

In [10]:
browser.visit(jpl_url)
browser.find_by_id('full_image').click()
featured_image_url = browser.find_by_css('.fancybox-image').first['src']
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17474_ip.jpg


#### Navigate the JPL site using BeautifulSoup to find the current Featured Image

In [11]:
browser.visit(jpl_url)
html = browser.html
soup = bs(html, 'html.parser')
result = soup.find('ul', class_='articles')
image = result.find_all('li')
for i in image:
    url = i.find('a')['data-fancybox-href']
featured_image_url = 'http://jpl.nasa.gov' + url
print(featured_image_url)

http://jpl.nasa.gov/spaceimages/images/largesize/PIA22588_hires.jpg


#### Scrape the latest Mars Weather tweet using requests

In [12]:
weather_response = requests.get(weather_url)
soup = bs(weather_response.text, 'html.parser')

In [13]:
weather_tweet = soup.find('p', class_='tweet-text').text.strip()
print(weather_tweet)

A view of this morning’s launch of the Parker #SolarProbe launch from about 3 miles awayhttps://youtu.be/hDhIane1Rso


#### Scrape the table containing facts from the Mars Facts webpage using Pandas

In [14]:
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [15]:
facts_df = tables[0]
facts_df.columns = ['Parameters','Values']
facts_df.set_index('Parameters', inplace=True)
facts_df

,Values
Parameters,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
facts_table = facts_df.to_html()
facts_table = facts_table.replace('\n', ' ')
print(facts_table)

<table border="1" class="dataframe">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Values</th>     </tr>     <tr>       <th>Parameters</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>


#### Scrape high resolution images of Mars hemispheres from USGS Astrogeology Site using BeautifulSoup

In [17]:
browser.visit(usgs_url)
html = browser.html
soup = bs(html, 'html.parser')
#print(soup.prettify())

In [18]:
results = soup.find('div', class_='result-list')                   
items = results.find_all('div', class_='item')                        
titles_urls = []

for item in items:                                              
    titles = item.find('div', class_='description')
    img_title = titles.a.text                                                
    img_title = img_title.replace(' Enhanced', '')
    browser.click_link_by_partial_text(img_title)                           
    
    html = browser.html                                                 
    soup = bs(html, 'html.parser')                                 
    
    linkss = soup.find('div', class_='downloads')
    links = linkss.find('ul')
    link = links.find('li')  
    img_url = link.a['href']
    
    titles_urls.append({'img_title': img_title, 'img_url': img_url})  
    
    browser.click_link_by_partial_text('Back')  

In [19]:
print(titles_urls)

[{'img_title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'img_title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'img_title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'img_title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
